In [1]:
from peewee import *
import json

In [2]:
db = SqliteDatabase('coal-db.sqlite')

In [3]:
operation_province_constraints = (
    "Aceh",
    "Bali",
    "Kepulauan Bangka Belitung",
    "Banten",
    "Bengkulu",
    "Gorontalo",
    "Papua Barat",
    "Jakarta",
    "Jambi",
    "Jawa Barat",
    "Jawa Tengah",
    "Jawa Timur",
    "Kalimantan Barat",
    "Kalimantan Selatan",
    "Kalimantan Tengah",
    "Kalimantan Timur",
    "Kalimantan Utara",
    "Kepulauan Riau",
    "Lampung",
    "Maluku Utara",
    "Maluku",
    "Nusa Tenggara Barat",
    "Nusa Tenggara Timur",
    "Papua",
    "Riau",
    "Sulawesi Barat",
    "Sulawesi Selatan",
    "Sulawesi Tengah",
    "Sulawesi Tenggara",
    "Sulawesi Utara",
    "Sumatera Barat",
    "Sumatera Selatan",
    "Sumatera Utara",
    "Yogyakarta"
)
company_type_constraints = (
    'Holding', 
    'Mine Owner', 
    'Consultant', 
    'Contractor', 
    'Trader', 
)
key_operation_constraints = (
    'Mining', 
    'Mining Services', 
    'Equipment Rental', 
    'Logistic Management', 
    'Overburden Removal & Hauling', 
    'Dredging', 
    'Coal Trading', 
    'Investment' 
)
mineral_type_constraints = (
    "Coal", 
    "Aluminium", 
    "Gold", 
    "Zinc and Lead", 
    "Oil", 
    "Nickel"
)

In [4]:
class Company(Model):
    id = IntegerField(primary_key=True)
    name = CharField()
    idx_ticker = CharField(null=True)
    operation_province = CharField(
        null=True,
        constraints=[
            Check(f"operation_province IN {operation_province_constraints}")
        ]
    )
    operation_kabkot = CharField(null=True)
    representative_address = TextField(null=True)
    company_type = CharField(
        null=True,
        constraints=[
            Check(f"company_type IN {company_type_constraints}")
        ]
    )
    key_operation = CharField(
        constraints=[
            Check(f"key_operation IN {key_operation_constraints}")
        ]
    )
    activities = TextField(
        null=True,
        constraints=[Check("json_valid(activities)")]
    )
    website = CharField(null=True)
    phone_number = IntegerField(null=True)
    email = CharField(null=True)

    class Meta:
        database = db
        table_name = 'company'

In [5]:

class CoalCompanyPerformance(Model):
    id = IntegerField(primary_key=True)
    company = ForeignKeyField(
        Company,
        column_name='company_id'
    )
    year = IntegerField(null=True)
    mineral_type = CharField(
        null=True,
        constraints=[
            Check(f"mineral_type IN {mineral_type_constraints}")
        ]
    )
    calorific_value = CharField(null=True)
    mining_operation_status = CharField(
        null=True,
        constraints=[
            Check("mining_operation_status IN ('production', 'development', 'inactive')")
        ]
    )
    mining_permit = CharField(
        null=True,
        constraints=[
            Check("mining_permit IN ('IUP', 'IUPK', 'PKP2B', 'IUPJ', 'WIUP')")
        ]
    )
    area = IntegerField(null=True)
    production_volume = FloatField(null=True)
    sales_volume = FloatField(null=True)
    overburden_removal_volume = FloatField(null=True)
    strip_ratio = FloatField(null=True)
    reserve = FloatField(null=True)
    resource = FloatField(null=True)
    mineral_sub_type = CharField(null=True)
    area_geometry = TextField(
        null=True,
        constraints=[Check("json_valid(area_geometry)")]
    )

    class Meta:
        database = db
        table_name = 'coal_company_performance'


In [6]:
class MiningSite(Model):
    id = IntegerField(primary_key=True)
    name = CharField()
    year = IntegerField(null=True)
    company = ForeignKeyField(
        Company,
        column_name='company_id'
    )
    calorific_value = CharField(null=True)
    production_volume = FloatField(null=True)
    overburden_removal_volume = FloatField(null=True)
    strip_ratio = FloatField(null=True)
    reserve = FloatField(null=True)
    resource = FloatField(null=True)
    province = CharField(
        null=True,
        constraints=[
            Check(f"operation_province IN {operation_province_constraints}")
        ]
    )
    city = CharField(null=True)
    mineral_type = CharField(
        null=True,
        constraints=[
            Check(f"mineral_type IN {mineral_type_constraints}")
        ]
    )
    class Meta:
        database = db
        table_name = 'mining_site'

In [7]:
companies = Company.select()
for company in companies:
    pass

ccp = CoalCompanyPerformance.select()
for c in ccp:
    pass

ms = MiningSite.select()
for m in ms:
    pass

In [8]:
class CompanyNew(Model):
    id = IntegerField(primary_key=True)
    name = CharField()
    idx_ticker = CharField(null=True)
    operation_province = CharField(
        null=True,
        constraints=[
            Check(f"operation_province IN {operation_province_constraints}")
        ]
    )
    operation_kabkot = CharField(null=True)
    representative_address = TextField(null=True)
    company_type = CharField(
        null=True,
        constraints=[
            Check(f"company_type IN {company_type_constraints}")
        ]
    )
    key_operation = CharField(
        constraints=[
            Check(f"key_operation IN {key_operation_constraints}")
        ]
    )
    activities = TextField(
        null=True,
        constraints=[Check("json_valid(activities)")]
    )
    website = CharField(null=True)
    phone_number = IntegerField(null=True)
    email = CharField(null=True)

    class Meta:
        database = db
        table_name = 'company_new'

CompanyNew.create_table()

with db.atomic():
    for company in Company.select():
        CompanyNew.create(
            id=company.id,
            name=company.name,
            idx_ticker=company.idx_ticker,
            operation_province=company.operation_province,
            operation_kabkot=company.operation_kabkot,
            representative_address=company.representative_address,
            company_type=company.company_type,
            key_operation=company.key_operation,
            activities=company.activities,
            website=company.website,
            phone_number=company.phone_number,
            email=company.email
        )

    # Drop the old table
    Company.drop_table()

    # Rename new table
    db.execute_sql('ALTER TABLE company_new RENAME TO company;')

In [9]:
class CoalCompanyPerformanceNew(Model):
    id = IntegerField(primary_key=True)
    company = ForeignKeyField(
        Company,
        column_name='company_id'
    )
    year = IntegerField(null=True)
    mineral_type = CharField(
        null=True,
        constraints=[
            Check(f"mineral_type IN {mineral_type_constraints}")
        ]
    )
    calorific_value = CharField(null=True)
    mining_operation_status = CharField(
        null=True,
        constraints=[
            Check("mining_operation_status IN ('production', 'development', 'inactive')")
        ]
    )
    mining_permit = CharField(
        null=True,
        constraints=[
            Check("mining_permit IN ('IUP', 'IUPK', 'PKP2B', 'IUPJ', 'WIUP')")
        ]
    )
    area = IntegerField(null=True)
    production_volume = FloatField(null=True)
    sales_volume = FloatField(null=True)
    overburden_removal_volume = FloatField(null=True)
    strip_ratio = FloatField(null=True)
    reserve = FloatField(null=True)
    resource = FloatField(null=True)
    mineral_sub_type = CharField(null=True)
    area_geometry = TextField(
        null=True,
        constraints=[Check("json_valid(area_geometry)")]
    )

    class Meta:
        database = db
        table_name = 'coal_company_performance_new'

# Create the new table
CoalCompanyPerformanceNew.create_table()

# Copy the data from the old table to the new table
with db.atomic():
    for record in CoalCompanyPerformance.select():
        CoalCompanyPerformanceNew.create(
            id=record.id,
            company=record.company,
            year=record.year,
            mineral_type=record.mineral_type,
            calorific_value=record.calorific_value,
            mining_operation_status=record.mining_operation_status,
            mining_permit=record.mining_permit,
            area=record.area,
            production_volume=record.production_volume,
            sales_volume=record.sales_volume,
            overburden_removal_volume=record.overburden_removal_volume,
            strip_ratio=record.strip_ratio,
            reserve=record.reserve,
            resource=record.resource,
            mineral_sub_type=record.mineral_sub_type,
            area_geometry=record.area_geometry
        )

    # Drop the old table
    CoalCompanyPerformance.drop_table()

    # Rename the new table to the original table name
    db.execute_sql('ALTER TABLE coal_company_performance_new RENAME TO coal_company_performance;')


In [11]:
class MiningSiteNew(Model):
    id = IntegerField(primary_key=True)
    name = CharField()
    year = IntegerField(null=True)
    company = ForeignKeyField(
        Company,
        column_name='company_id'
    )
    calorific_value = CharField(null=True)
    production_volume = FloatField(null=True)
    overburden_removal_volume = FloatField(null=True)
    strip_ratio = FloatField(null=True)
    reserve = FloatField(null=True)
    resource = FloatField(null=True)
    province = CharField(
        null=True,
        constraints=[
            Check(f"province IN {operation_province_constraints}")
        ]
    )
    city = CharField(null=True)
    mineral_type = CharField(
        null=True,
        constraints=[
            Check(f"mineral_type IN {mineral_type_constraints}")
        ]
    )

    class Meta:
        database = db
        table_name = 'mining_site_new'

MiningSiteNew.create_table()

with db.atomic():
    for ms in MiningSite.select():
        MiningSiteNew.create(
            id=ms.id,
            name=ms.name,
            year=ms.year,
            company=ms.company,
            calorific_value=ms.calorific_value,
            production_volume=ms.production_volume,
            overburden_removal_volume=ms.overburden_removal_volume,
            strip_ratio=ms.overburden_removal_volume,
            reserve=ms.reserve,
            resource=ms.resource,
            province=ms.province,
            city=ms.city,
            mineral_type=ms.mineral_type
        )

    # Drop the old table
    MiningSite.drop_table()

    # Rename new table
    db.execute_sql('ALTER TABLE mining_site_new RENAME TO mining_site;')